In [ ]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [1]'.</span>

An Exception was encountered at 'In [1]'.

# Working with SAS

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
  

## SAS <a id="SAS"></a>

A SAS dataset is roughly equivalent to Python's DataFrame (Pandas) and R's dataframe, and this is the only datatype that can be exchanged between SAS and other languages.

For example, you can get a R dataframe into SAS as follows:

In [1]:
%get mtcars --from R
PROC PRINT;
RUN;

ValueError: No subkernel named SAS is found. Please make sure that you have the kernel installed (listed in the output of "jupyter kernelspec list" and usable in jupyter by itself), install appropriate language module (e.g. "pip install sos-r"), restart jupyter notebook and try again.

And if you have a SAS dataset,

In [ ]:
DATA CLASS;
     INPUT NAME $ 1-8 SEX $ 10 AGE 12-13 HEIGHT 15-16 WEIGHT 18-22;
CARDS;
JOHN     M 12 59 99.5
JAMES    M 12 57 83.0
ALFRED   M 14 69 112.5
ALICE    F 13 56 84.0
PROC PRINT;
RUN;

You can transfer the dataset to Python or R and analyze there

In [ ]:
%get CLASS --from SAS
%preview -n CLASS

## Further reading

* [How to exchange variables among living subkernels](doc/user_guide/exchange_variable.html)